In [35]:
from datasets import load_dataset, concatenate_datasets

In [2]:
cc_news = load_dataset('csv',data_files = "cc_news_ds.csv")

In [3]:
openweb = load_dataset('csv', data_files='openwebtext.csv')

Loading dataset shards:   0%|          | 0/74 [00:00<?, ?it/s]

In [13]:
wiki_ds = load_dataset('csv',data_files = "wiki_ds.csv")

Generating train split: 0 examples [00:00, ? examples/s]

Loading dataset shards:   0%|          | 0/39 [00:00<?, ?it/s]

In [9]:
cc_news

DatasetDict({
    train: Dataset({
        features: ['title', 'text', 'domain', 'date', 'description', 'url', 'image_url'],
        num_rows: 70824
    })
})

In [11]:
openweb

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 8013769
    })
})

In [15]:
wiki_ds

DatasetDict({
    train: Dataset({
        features: ['id', 'url', 'title', 'text'],
        num_rows: 6458670
    })
})

In [17]:
def preprocess(ex):
    text = ex.get("text") or ""
    if "title" in ex or "description" in ex:  # CC-News
        text = " ".join(filter(None, [ex.get("title"), ex.get("description"), ex.get("text")]))
    return {"text": text.strip()}


In [25]:
[c for c in wiki_ds['train'].column_names if c != "text"]

['id', 'url', 'title']

In [27]:
wiki_ds = wiki_ds['train'].map(preprocess, remove_columns=[c for c in wiki_ds['train'].column_names if c != "text"])
openweb = openweb['train'].map(preprocess, remove_columns=[c for c in openweb['train'].column_names if c != "text"])
cc_news = cc_news['train'].map(preprocess, remove_columns=[c for c in cc_news['train'].column_names if c != "text"])


Map:   0%|          | 0/6458670 [00:00<?, ? examples/s]

Map:   0%|          | 0/8013769 [00:00<?, ? examples/s]

Map:   0%|          | 0/70824 [00:00<?, ? examples/s]

In [29]:
wiki_ds

Dataset({
    features: ['text'],
    num_rows: 6458670
})

In [37]:
train_ds = concatenate_datasets([wiki_ds, openweb, cc_news])


In [39]:
train_ds = train_ds.filter(lambda x: len(x["text"]) > 100)

Filter:   0%|          | 0/14543263 [00:00<?, ? examples/s]

In [41]:
train_ds

Dataset({
    features: ['text'],
    num_rows: 14526347
})

In [43]:
train_ds.to_csv("training_data.csv")

Creating CSV from Arrow format:   0%|          | 0/14527 [00:00<?, ?ba/s]

59830529395

In [45]:
train_data = load_dataset('csv',data_files = "training_data.csv")

Generating train split: 0 examples [00:00, ? examples/s]

Loading dataset shards:   0%|          | 0/112 [00:00<?, ?it/s]

In [47]:
train_data

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 14526347
    })
})

In [49]:
#working on smaller dataset

import itertools
import random

# Shuffle and take first ~2 million articles
small_wiki = itertools.islice(wiki_ds, 2_000_000)


In [53]:
small_list = list(small_wiki)

KeyboardInterrupt: 

In [82]:
wiki_small = wiki_ds.shuffle(seed=42).select(range(1_000_000))

In [108]:
wiki_small.to_csv("wiki_small.csv")

Creating CSV from Arrow format:   0%|          | 0/1000 [00:00<?, ?ba/s]

3070402330

In [84]:
wiki_small

Dataset({
    features: ['text'],
    num_rows: 1000000
})

In [96]:
openweb_small = openweb.shuffle(seed=42).select(range(250_000))

In [98]:
wiki_small_500k = wiki_ds.shuffle(seed=42).select(range(250_000))

In [110]:
wiki_small_500k.to_csv("wiki_small_500k.csv")

Creating CSV from Arrow format:   0%|          | 0/250 [00:00<?, ?ba/s]

770923201

In [114]:
wiki_small_100k = wiki_ds.shuffle(seed=42).select(range(100_000))

In [116]:
wiki_small_100k.to_csv("wiki_small_100k.csv")

Creating CSV from Arrow format:   0%|          | 0/100 [00:00<?, ?ba/s]

309043293

In [118]:
wiki_small_50k = wiki_ds.shuffle(seed=42).select(range(50_000))

In [120]:
wiki_small_50k.to_csv("wiki_small_50k.csv")

Creating CSV from Arrow format:   0%|          | 0/50 [00:00<?, ?ba/s]

153916972

In [74]:
openweb_small

Dataset({
    features: ['text'],
    num_rows: 1000000
})

In [112]:
wiki_small_500k

Dataset({
    features: ['text'],
    num_rows: 250000
})

In [76]:
cc_news

Dataset({
    features: ['text'],
    num_rows: 70824
})

In [100]:
train_ds_small = concatenate_datasets([wiki_small_500k, openweb_small, cc_news])


In [106]:
train_ds_small.to_csv("train_ds_small.csv")

Creating CSV from Arrow format:   0%|          | 0/571 [00:00<?, ?ba/s]

2183957799

In [104]:
train_ds_small

Dataset({
    features: ['text'],
    num_rows: 570824
})